# Cricket Match lineups

In [69]:
import numpy as np
import pandas as pd
import json
import http.client
import time  
# List of match IDs

match_ids = [
    13265827, 13265828, 13265829, 13265830, 13265831, 13265832, 13265833, 
    13265834, 13265836, 13265837, 13265838, 13265840, 13558575, 13558576, 13569002
]

# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}

# Parent dictionary to store all match data
parent_json = {}

# Loop through each match ID
for match_id in match_ids:
    try:
        # Make API request
        conn.request("GET", f"/api/cricket/match/{match_id}/lineups", headers=headers)
        res = conn.getresponse()
        data = res.read()
        
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        
        # Store JSON in the parent dictionary with match_id as key
        parent_json[f"match_lineups_{match_id}"] = data_json

        print(f"Data collected for Match ID {match_id}")

    except Exception as e:
        print(f"Error fetching data for Match ID {match_id}: {e}")

    time.sleep(1)  # Optional: Delay to prevent rate limits

# Save all match data in a single JSON file
with open("all_match_lineups_data.json", "w", encoding="utf-8") as f:
    json.dump(parent_json, f, indent=4)

print("All match data saved in all_match_lineups_data.json")

Data collected for Match ID 13265827
Data collected for Match ID 13265828
Data collected for Match ID 13265829
Data collected for Match ID 13265830
Data collected for Match ID 13265831
Data collected for Match ID 13265832
Error fetching data for Match ID 13265833: Expecting value: line 1 column 1 (char 0)
Data collected for Match ID 13265834
Error fetching data for Match ID 13265836: Expecting value: line 1 column 1 (char 0)
Data collected for Match ID 13265837
Data collected for Match ID 13265838
Data collected for Match ID 13265840
Data collected for Match ID 13558575
Data collected for Match ID 13558576
Data collected for Match ID 13569002
All match data saved in all_match_lineups_data.json


## Loading Json file

In [1]:
import numpy as np
import pandas as pd
import json
with open("all_match_lineups_data.json", "r", encoding="utf-8") as f:
    all_match_lineups = json.load(f)
all_match_lineups

{'match_lineups_13265827': {'confirmed': True,
  'home': {'players': [{'player': {'name': 'Abrar Ahmed',
      'slug': 'abrar-ahmed',
      'shortName': 'A Ahmed',
      'position': 'B',
      'userCount': 33,
      'id': 871548,
      'country': {'alpha2': 'PK',
       'alpha3': 'PAK',
       'name': 'Pakistan',
       'slug': 'pakistan'},
      'marketValueCurrency': 'EUR',
      'dateOfBirthTimestamp': 908496000,
      'cricketPlayerInfo': {'batting': 'Right', 'bowling': 'Leg break googly'},
      'fieldTranslations': {'nameTranslation': {'ar': 'أبرار أحمد',
        'hi': 'अबरार अहमद',
        'bn': 'আবরার আহমেদ'},
       'shortNameTranslation': {'ar': 'أبرار أحمد',
        'hi': 'अबरार अहमद',
        'bn': 'আবরার আহমেদ'}}},
     'teamId': 212417,
     'position': 'B',
     'substitute': False},
    {'player': {'name': 'Babar Azam',
      'slug': 'babar-azam',
      'shortName': 'Babar Azam',
      'position': 'BM',
      'jerseyNumber': '56',
      'height': 180,
      'userCount':

In [ ]:
all_match_lineups['match_lineups_13265827']

{'confirmed': True,
 'home': {'players': [{'player': {'name': 'Axar Patel',
     'slug': 'axar-patel',
     'shortName': 'A Patel',
     'position': 'AR',
     'jerseyNumber': '20',
     'height': 180,
     'userCount': 244,
     'id': 1198740,
     'country': {'alpha2': 'IN',
      'alpha3': 'IND',
      'name': 'India',
      'slug': 'india'},
     'marketValueCurrency': 'EUR',
     'dateOfBirthTimestamp': 759024000,
     'cricketPlayerInfo': {'batting': 'Left',
      'bowling': 'Slow left-arm orthodox'},
     'fieldTranslations': {'nameTranslation': {'ar': 'أكسار باتيل',
       'hi': 'अक्षर पटेल',
       'bn': 'অক্ষর প্যাটেল'},
      'shortNameTranslation': {'ar': 'أكسار باتيل',
       'hi': 'अक्षर पटेल',
       'bn': 'অক্ষর প্যাটেল'}}},
    'teamId': 187841,
    'position': 'AR',
    'substitute': False},
   {'player': {'name': 'Hardik Pandya',
     'slug': 'hardik-pandya',
     'shortName': 'Hardik Pandya',
     'position': 'AR',
     'jerseyNumber': '33',
     'userCount': 1936,


# Depth json to flatten_json

In [4]:
# Function for Convert a nested json dictionary to flatten json or dictionary
def flatten_json(nested_json, parent_key='', sep='_'):

    items = []
    
    # If the value is a dictionary
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key
            items.extend(flatten_json(value, new_key, sep=sep).items())
    
    # If the value is a list
    elif isinstance(nested_json, list): 
        for index, item in enumerate(nested_json):
            new_key = f"{parent_key}{sep}{index}" if parent_key else str(index)
            items.extend(flatten_json(item, new_key, sep=sep).items())
            
    # If the value is neither dict nor list
    else:
        items.append((parent_key, nested_json))

    return dict(items)

# Flatten Json to Row column

In [5]:
def process_json_file(data):
    try:
        # Normalize JSON data
        if isinstance(data, list):  # JSON is an array of objects
            flat_data = [flatten_json(item) for item in data]
        elif isinstance(data, dict):  # JSON is a single dictionary
            flat_data = [flatten_json(data)]
        else:
            raise ValueError("Unsupported JSON format")

        # Convert to DataFrame
        df = pd.DataFrame(flat_data)
        return df


    except json.JSONDecodeError:
        print("Error: Invalid JSON file.")
    except Exception as e:
        print(f"Error: {str(e)}")

# Extracting Home team Players data

In [11]:
for i in data['home']['players']:
    print(i)

{'player': {'name': 'Axar Patel', 'slug': 'axar-patel', 'shortName': 'A Patel', 'position': 'AR', 'jerseyNumber': '20', 'height': 180, 'userCount': 232, 'id': 1198740, 'country': {'alpha2': 'IN', 'alpha3': 'IND', 'name': 'India', 'slug': 'india'}, 'marketValueCurrency': 'EUR', 'dateOfBirthTimestamp': 759024000, 'cricketPlayerInfo': {'batting': 'Left', 'bowling': 'Slow left-arm orthodox'}, 'fieldTranslations': {'nameTranslation': {'ar': 'أكسار باتيل', 'hi': 'अक्षर पटेल', 'bn': 'অক্ষর প্যাটেল'}, 'shortNameTranslation': {'ar': 'أكسار باتيل', 'hi': 'अक्षर पटेल', 'bn': 'অক্ষর প্যাটেল'}}}, 'teamId': 187841, 'position': 'AR', 'substitute': False}
{'player': {'name': 'Hardik Pandya', 'slug': 'hardik-pandya', 'shortName': 'H Pandya', 'position': 'AR', 'jerseyNumber': '33', 'userCount': 1824, 'id': 794290, 'country': {'alpha2': 'IN', 'alpha3': 'IND', 'name': 'India', 'slug': 'india'}, 'marketValueCurrency': 'EUR', 'dateOfBirthTimestamp': 750297600, 'cricketPlayerInfo': {'batting': 'Right', 'bowl

# Process my parent dict ot file

In [12]:
# For home

In [7]:
def process_dictionary(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] and data[key] != {}:  # Ensure the value is not empty
            if isinstance(data[key], list):  # If it's a list
                if len(data[key]) == 0: # Fix: Check if the list is empty
                    continue  # Skip empty lists to prevent IndexError

                first_item = flatten_json(data[key][0])
                dataframe = pd.DataFrame(columns=process_json_file(first_item).columns)

                for item in data[key]:
                    flatten_dict = flatten_json(item)
                    new_dataframe = process_json_file(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If it's a dictionary
                flatten_dict = flatten_json(data[key])
                if not flatten_dict:
                    continue  # Skip empty dictionaries

                dataframe = process_json_file(flatten_dict)
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    if parent_json:  # Only process if parent_json has data
        parent_df = process_json_file(flatten_json(parent_json))
        if parent_df is not None:
            all_dataframe["parent_df"] = parent_df

    return all_dataframe



In [19]:
all_cricket_match_lineups = pd.DataFrame()
all_cricket_match_lineups

""


In [20]:
all_match_lineups.keys()

dict_keys(['match_lineups_13265827', 'match_lineups_13265828', 'match_lineups_13265829', 'match_lineups_13265830', 'match_lineups_13265831', 'match_lineups_13265832', 'match_lineups_13265834', 'match_lineups_13265837', 'match_lineups_13265838', 'match_lineups_13265840', 'match_lineups_13558575', 'match_lineups_13558576', 'match_lineups_13569002'])

In [21]:
for i in all_match_lineups.keys():  
    match_id = int(i.split('_')[-1])
    
    # Home lineups details
    home_lineups = process_dictionary(data['home'])
    home_lineups['players']['match_id'] = match_id
    home_lineups['players']['innings'] = 'home'
    home_lineups['parent_df'] = home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 
    home_lineups_details = pd.concat([home_lineups['players'], home_lineups['parent_df']], axis=1)

    # Away lineups details
    away_lineups = process_dictionary(data['away'])
    away_lineups['players']['match_id'] = match_id
    away_lineups['players']['innings'] = 'away'
    away_lineups['parent_df'] = away_lineups['parent_df'].loc[away_lineups['parent_df'].index.repeat(len(away_lineups['players']))].reset_index(drop=True) 
    away_lineups_details = pd.concat([away_lineups['players'], away_lineups['parent_df']], axis=1)

    lineups_details = pd.concat([home_lineups_details, away_lineups_details], ignore_index=False)
    all_cricket_match_lineups = pd.concat([all_cricket_match_lineups, lineups_details], ignore_index=False)


In [22]:
all_cricket_match_lineups

,player_name,player_slug,player_shortName,player_position,player_userCount,player_id,player_country_alpha2,player_country_alpha3,player_country_name,player_country_slug,...,supportStaff_1_country_alpha3,supportStaff_1_country_name,supportStaff_1_country_slug,supportStaff_2_name,supportStaff_2_role,supportStaff_2_id,supportStaff_2_country_alpha2,supportStaff_2_country_alpha3,supportStaff_2_country_name,supportStaff_2_country_slug
0,Abrar Ahmed,abrar-ahmed,A Ahmed,B,33,871548,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Babar Azam,babar-azam,Babar Azam,BM,1714,794647,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fakhar Zaman,fakhar-zaman,Fakhar Zaman,BM,205,795320,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Haris Rauf,haris-rauf,Haris Rauf,B,104,975942,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Khushdil Shah,khushdil-shah,K Shah,AR,30,874504,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Mitchell Santner,mitchell-santner,Mitchell Santner,AR,96,786835,NZ,NZL,New Zealand,new-zealand,...,NZL,New Zealand,new-zealand,Gary Stead,coach,123.0,NZ,NZL,New Zealand,new-zealand
11,Jacob Duffy,jacob-duffy,J Duffy,B,6,915786,NZ,NZL,New Zealand,new-zealand,...,NZL,New Zealand,new-zealand,Gary Stead,coach,123.0,NZ,NZL,New Zealand,new-zealand
12,Kyle Jamieson,kyle-jamieson,K Jamieson,B,12,920980,NZ,NZL,New Zealand,new-zealand,...,NZL,New Zealand,new-zealand,Gary Stead,coach,123.0,NZ,NZL,New Zealand,new-zealand
13,Mark Chapman,mark-chapman,M Chapman,BM,19,787287,NZ,NZL,New Zealand,new-zealand,...,NZL,New Zealand,new-zealand,Gary Stead,coach,123.0,NZ,NZL,New Zealand,new-zealand


In [24]:
# saving all files
all_cricket_match_lineups.to_excel('all_cricket_match_lineups.xlsx', index=False)

In [11]:
data = all_match_lineups['match_lineups_13265827']
data

{'confirmed': True,
 'home': {'players': [{'player': {'name': 'Abrar Ahmed',
     'slug': 'abrar-ahmed',
     'shortName': 'A Ahmed',
     'position': 'B',
     'userCount': 33,
     'id': 871548,
     'country': {'alpha2': 'PK',
      'alpha3': 'PAK',
      'name': 'Pakistan',
      'slug': 'pakistan'},
     'marketValueCurrency': 'EUR',
     'dateOfBirthTimestamp': 908496000,
     'cricketPlayerInfo': {'batting': 'Right', 'bowling': 'Leg break googly'},
     'fieldTranslations': {'nameTranslation': {'ar': 'أبرار أحمد',
       'hi': 'अबरार अहमद',
       'bn': 'আবরার আহমেদ'},
      'shortNameTranslation': {'ar': 'أبرار أحمد',
       'hi': 'अबरार अहमद',
       'bn': 'আবরার আহমেদ'}}},
    'teamId': 212417,
    'position': 'B',
    'substitute': False},
   {'player': {'name': 'Babar Azam',
     'slug': 'babar-azam',
     'shortName': 'Babar Azam',
     'position': 'BM',
     'jerseyNumber': '56',
     'height': 180,
     'userCount': 1714,
     'id': 794647,
     'country': {'alpha2': 'PK

In [12]:
all_cricket_match_lineups = pd.DataFrame()
all_cricket_match_lineups

""


In [ ]:

# Home lineups details
match_id = int('match_odds_13569002'.split('_')[-1])
home_lineups = process_dictionary(data['home'])
home_lineups['players']['match_id'] = match_id
home_lineups['players']['innings'] = 'home'
home_lineups['parent_df'] = home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 
home_lineups_details = pd.concat([home_lineups['players'], home_lineups['parent_df']], axis=1)

# Away lineups details
away_lineups = process_dictionary(data['away'])
away_lineups['players']['match_id'] = match_id
away_lineups['players']['innings'] = 'away'
away_lineups['parent_df'] = away_lineups['parent_df'].loc[away_lineups['parent_df'].index.repeat(len(away_lineups['players']))].reset_index(drop=True) 
away_lineups_details = pd.concat([away_lineups['players'], away_lineups['parent_df']], axis=1)

pd.concat([home_lineups_details, away_lineups_details], ignore_index=False)

,player_name,player_slug,player_shortName,player_position,player_userCount,player_id,player_country_alpha2,player_country_alpha3,player_country_name,player_country_slug,...,supportStaff_1_country_alpha3,supportStaff_1_country_name,supportStaff_1_country_slug,supportStaff_2_name,supportStaff_2_role,supportStaff_2_id,supportStaff_2_country_alpha2,supportStaff_2_country_alpha3,supportStaff_2_country_name,supportStaff_2_country_slug
0,Abrar Ahmed,abrar-ahmed,A Ahmed,B,33,871548,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Babar Azam,babar-azam,Babar Azam,BM,1714,794647,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fakhar Zaman,fakhar-zaman,Fakhar Zaman,BM,205,795320,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Haris Rauf,haris-rauf,Haris Rauf,B,104,975942,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Khushdil Shah,khushdil-shah,K Shah,AR,30,874504,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Naseem Shah,naseem-shah,Naseem Shah,B,222,956739,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Salman Agha,agha-salman,S Agha,AR,69,786840,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Saud Shakeel,saud-shakeel,S Shakeel,BM,29,975996,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Shaheen Afridi,shaheen-afridi,S Afridi,B,393,911674,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tayyab Tahir,tayyab-tahir,T Tahir,BM,12,1212744,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df = pd.concat([all_cricket_match_lineups, home_lineups_details])
df

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
1,Hardik Pandya,hardik-pandya,H Pandya,AR,33,NaN,1824,794290,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
2,Kuldeep Yadav,kuldeep-yadav,K Yadav,B,23,NaN,349,786807,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
3,Mohammad Shami,mohammed-shami,M Shami,B,11,NaN,227,786526,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
4,Ravindra Jadeja,ravindra-jadeja,R Jadeja,AR,8,173,835,786779,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
5,Shreyas Iyer,shreyas-iyer,Shreyas Iyer,BM,96,178,538,793972,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
6,Shubman Gill,shubman-gill,Shubman Gill,BM,77,185,1695,932039,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
7,Varun Chakaravarthy,varun-chakaravarthy,Varun Chakaravarthy,B,NaN,NaN,101,1198753,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
8,Virat Kohli,virat-kohli,V Kohli,BM,18,175,5586,786521,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
9,KL Rahul,kl-rahul,Rahul,WK,1,180,782,786519,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [ ]:
# THe main thing is to gathert

In [59]:
away_lineups = process_dictionary(data['away'])

away_lineups['players']['match_id'] = int('match_odds_13569002'.split('_')[-1])
away_lineups['players']['innings'] = 'away'
away_lineups['parent_df'] = away_lineups['parent_df'].loc[away_lineups['parent_df'].index.repeat(len(away_lineups['players']))].reset_index(drop=True) 
away_lineups_details = pd.concat([away_lineups['players'], away_lineups['parent_df']], axis=1)
away_lineups_details

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,supportStaff_2_country_name,supportStaff_2_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Daryl Mitchell,daryl-mitchell,Daryl Mitchell,AR,75,183,98,786833,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
1,Glenn Phillips,glenn-phillips,Glenn Phillips,AR,23,NaN,106,872201,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
2,Kane Williamson,kane-williamson,K Williamson,BM,22,174,290,786552,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
3,Kyle Jamieson,kyle-jamieson,Kyle Jamieson,B,12,203,11,920980,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
4,Michael Bracewell,michael-bracewell,Michael Bracewell,AR,4,178,15,915980,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
5,Nathan Smith,nathan-smith,N Smith,AR,NaN,NaN,10,967474,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
6,Rachin Ravindra,rachin-ravindra,R Ravindra,AR,8,NaN,253,974426,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
7,Will O'Rourke,william-orourke,Will O'Rourke,B,NaN,NaN,13,1212990,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
8,Will Young,will-young,W Young,BM,32,188,32,916142,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226
9,Tom Latham,tom-latham,T Latham,WK,48,173,31,786550,NZ,NZL,...,New Zealand,new-zealand,000000,222226,808080,222226,000000,222226,808080,222226


In [66]:
pd.concat([df, away_lineups_details], ignore_index=True)

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,supportStaff_1_country_alpha3,supportStaff_1_country_name,supportStaff_1_country_slug,supportStaff_2_name,supportStaff_2_role,supportStaff_2_id,supportStaff_2_country_alpha2,supportStaff_2_country_alpha3,supportStaff_2_country_name,supportStaff_2_country_slug
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hardik Pandya,hardik-pandya,H Pandya,AR,33,NaN,1824,794290,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kuldeep Yadav,kuldeep-yadav,K Yadav,B,23,NaN,349,786807,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mohammad Shami,mohammed-shami,M Shami,B,11,NaN,227,786526,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ravindra Jadeja,ravindra-jadeja,R Jadeja,AR,8,173,835,786779,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Shreyas Iyer,shreyas-iyer,Shreyas Iyer,BM,96,178,538,793972,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Shubman Gill,shubman-gill,Shubman Gill,BM,77,185,1695,932039,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Varun Chakaravarthy,varun-chakaravarthy,Varun Chakaravarthy,B,NaN,NaN,101,1198753,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Virat Kohli,virat-kohli,V Kohli,BM,18,175,5586,786521,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KL Rahul,kl-rahul,Rahul,WK,1,180,782,786519,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
away_lineups_details.to_excel('away_lineups_details.xlsx', index=False)

In [ ]:
home_lineups['parent_df'] = home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [47]:
home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
1,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
2,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
3,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
4,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
5,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
6,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
7,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
8,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
9,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [44]:
home_lineups['players']

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,player_marketValueCurrency,player_dateOfBirthTimestamp,player_cricketPlayerInfo_batting,player_cricketPlayerInfo_bowling,teamId,position,substitute,player_firstName,player_lastName,captain
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,EUR,759024000,Left,Slow left-arm orthodox,187841,AR,False,NaN,NaN,NaN
1,Hardik Pandya,hardik-pandya,H Pandya,AR,33,NaN,1824,794290,IN,IND,...,EUR,750297600,Right,Right-arm medium fast,187780,AR,False,NaN,NaN,NaN
2,Kuldeep Yadav,kuldeep-yadav,K Yadav,B,23,NaN,349,786807,IN,IND,...,EUR,787363200,Left,Left wrist spin,187841,B,False,NaN,NaN,NaN
3,Mohammad Shami,mohammed-shami,M Shami,B,11,NaN,227,786526,IN,IND,...,EUR,652320000,Right,Right-arm fast,187844,B,False,,,NaN
4,Ravindra Jadeja,ravindra-jadeja,R Jadeja,AR,8,173,835,786779,IN,IND,...,EUR,597369600,Left,Slow left-arm orthodox,187777,AR,False,NaN,NaN,NaN
5,Shreyas Iyer,shreyas-iyer,Shreyas Iyer,BM,96,178,538,793972,IN,IND,...,EUR,786672000,Right,Leg break googly,187776,BM,False,NaN,NaN,NaN
6,Shubman Gill,shubman-gill,Shubman Gill,BM,77,185,1695,932039,IN,IND,...,EUR,936748800,Right,Off break,216006,BM,False,NaN,NaN,NaN
7,Varun Chakaravarthy,varun-chakaravarthy,Varun Chakaravarthy,B,NaN,NaN,101,1198753,IN,IND,...,EUR,683424000,Right,Leg break googly,187775,B,False,NaN,NaN,NaN
8,Virat Kohli,virat-kohli,V Kohli,BM,18,175,5586,786521,IN,IND,...,EUR,594691200,Right,Right-arm medium,187843,BM,False,NaN,NaN,NaN
9,KL Rahul,kl-rahul,Rahul,WK,1,180,782,786519,IN,IND,...,EUR,703555200,Right,NaN,187841,WK,False,,,NaN


In [45]:
home_lineups['parent_df']

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [46]:
pd.concat([home_lineups['players'], home_lineups['parent_df']], axis=1)

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226
1,Hardik Pandya,hardik-pandya,H Pandya,AR,33,NaN,1824,794290,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kuldeep Yadav,kuldeep-yadav,K Yadav,B,23,NaN,349,786807,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mohammad Shami,mohammed-shami,M Shami,B,11,NaN,227,786526,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ravindra Jadeja,ravindra-jadeja,R Jadeja,AR,8,173,835,786779,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Shreyas Iyer,shreyas-iyer,Shreyas Iyer,BM,96,178,538,793972,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Shubman Gill,shubman-gill,Shubman Gill,BM,77,185,1695,932039,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Varun Chakaravarthy,varun-chakaravarthy,Varun Chakaravarthy,B,NaN,NaN,101,1198753,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Virat Kohli,virat-kohli,V Kohli,BM,18,175,5586,786521,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,KL Rahul,kl-rahul,Rahul,WK,1,180,782,786519,IN,IND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
home_lineups['parent_df']

,players_0_player_name,players_0_player_slug,players_0_player_shortName,players_0_player_position,players_0_player_jerseyNumber,players_0_player_height,players_0_player_userCount,players_0_player_id,players_0_player_country_alpha2,players_0_player_country_alpha3,...,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [ ]:
home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber
0,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
1,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
2,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
3,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
4,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
5,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
6,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
7,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
8,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226
9,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226


In [15]:
all_dataframe = process_dictionary(data['home'])
all_dataframe

{'players':             player_name          player_slug     player_shortName  \
 0            Axar Patel           axar-patel              A Patel   
 1         Hardik Pandya        hardik-pandya             H Pandya   
 2         Kuldeep Yadav        kuldeep-yadav              K Yadav   
 3        Mohammad Shami       mohammed-shami              M Shami   
 4       Ravindra Jadeja      ravindra-jadeja             R Jadeja   
 5          Shreyas Iyer         shreyas-iyer         Shreyas Iyer   
 6          Shubman Gill         shubman-gill         Shubman Gill   
 7   Varun Chakaravarthy  varun-chakaravarthy  Varun Chakaravarthy   
 8           Virat Kohli          virat-kohli              V Kohli   
 9              KL Rahul             kl-rahul                Rahul   
 10         Rohit Sharma         rohit-sharma             R Sharma   
 11       Arshdeep Singh       arshdeep-singh       Arshdeep Singh   
 12         Harshit Rana         harshit-rana         Harshit Rana   
 13      

In [16]:
# creating match_id feature in every dataframe
for key in all_dataframe:
    all_dataframe[key]['match_id'] = match_id
    all_dataframe[key]['team'] = 'home'

In [17]:
all_dataframe['players']

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,player_cricketPlayerInfo_batting,player_cricketPlayerInfo_bowling,teamId,position,substitute,player_firstName,player_lastName,captain,match_id,team
0,Axar Patel,axar-patel,A Patel,AR,20,180,232,1198740,IN,IND,...,Left,Slow left-arm orthodox,187841,AR,False,NaN,NaN,NaN,13569002,home
1,Hardik Pandya,hardik-pandya,H Pandya,AR,33,NaN,1824,794290,IN,IND,...,Right,Right-arm medium fast,187780,AR,False,NaN,NaN,NaN,13569002,home
2,Kuldeep Yadav,kuldeep-yadav,K Yadav,B,23,NaN,349,786807,IN,IND,...,Left,Left wrist spin,187841,B,False,NaN,NaN,NaN,13569002,home
3,Mohammad Shami,mohammed-shami,M Shami,B,11,NaN,227,786526,IN,IND,...,Right,Right-arm fast,187844,B,False,,,NaN,13569002,home
4,Ravindra Jadeja,ravindra-jadeja,R Jadeja,AR,8,173,835,786779,IN,IND,...,Left,Slow left-arm orthodox,187777,AR,False,NaN,NaN,NaN,13569002,home
5,Shreyas Iyer,shreyas-iyer,Shreyas Iyer,BM,96,178,538,793972,IN,IND,...,Right,Leg break googly,187776,BM,False,NaN,NaN,NaN,13569002,home
6,Shubman Gill,shubman-gill,Shubman Gill,BM,77,185,1695,932039,IN,IND,...,Right,Off break,216006,BM,False,NaN,NaN,NaN,13569002,home
7,Varun Chakaravarthy,varun-chakaravarthy,Varun Chakaravarthy,B,NaN,NaN,101,1198753,IN,IND,...,Right,Leg break googly,187775,B,False,NaN,NaN,NaN,13569002,home
8,Virat Kohli,virat-kohli,V Kohli,BM,18,175,5586,786521,IN,IND,...,Right,Right-arm medium,187843,BM,False,NaN,NaN,NaN,13569002,home
9,KL Rahul,kl-rahul,Rahul,WK,1,180,782,786519,IN,IND,...,Right,NaN,187841,WK,False,,,NaN,13569002,home


In [18]:
all_dataframe['parent_df']

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber,match_id,team
0,Gautam Gambhir,coach,157,IN,IND,India,india,000000,222226,808080,222226,000000,222226,808080,222226,13569002,home


# Saving file

In [19]:
output_file = "home.xlsx"  # Define output Excel file

for i, (key, df) in enumerate(all_dataframe.items()):
    print(f"Saving data for {key} in {output_file}")
    
    # If it's the first DataFrame, write with mode 'w' (write mode) to create the file
    # For the rest, use mode 'a' (append) without re-writing the file
    with pd.ExcelWriter(output_file, engine="openpyxl", mode='w' if i == 0 else 'a') as writer:
        df.to_excel(writer, sheet_name=key[:31], index=False)  # Ensure sheet name is within 31 chars

print(f"All DataFrames have been saved in {output_file}")

Saving data for players in home.xlsx
Saving data for parent_df in home.xlsx
All DataFrames have been saved in home.xlsx


# For away

In [20]:
def process_dictionary(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] and data[key] != {}:  # Ensure the value is not empty
            if isinstance(data[key], list):  # If it's a list
                if len(data[key]) == 0: # Fix: Check if the list is empty
                    continue  # Skip empty lists to prevent IndexError

                first_item = flatten_json(data[key][0])
                dataframe = pd.DataFrame(columns=process_json_file(first_item).columns)

                for item in data[key]:
                    flatten_dict = flatten_json(item)
                    new_dataframe = process_json_file(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If it's a dictionary
                flatten_dict = flatten_json(data[key])
                if not flatten_dict:
                    continue  # Skip empty dictionaries

                dataframe = process_json_file(flatten_dict)
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    if parent_json:  # Only process if parent_json has data
        parent_df = process_json_file(flatten_json(parent_json))
        if parent_df is not None:
            all_dataframe["parent_df"] = parent_df

    return all_dataframe



In [21]:
all_dataframe = process_dictionary(data['away'])
all_dataframe

{'players':           player_name        player_slug   player_shortName player_position  \
 0      Daryl Mitchell     daryl-mitchell     Daryl Mitchell              AR   
 1      Glenn Phillips     glenn-phillips     Glenn Phillips              AR   
 2     Kane Williamson    kane-williamson       K Williamson              BM   
 3       Kyle Jamieson      kyle-jamieson      Kyle Jamieson               B   
 4   Michael Bracewell  michael-bracewell  Michael Bracewell              AR   
 5        Nathan Smith       nathan-smith            N Smith              AR   
 6     Rachin Ravindra    rachin-ravindra         R Ravindra              AR   
 7       Will O'Rourke    william-orourke      Will O'Rourke               B   
 8          Will Young         will-young            W Young              BM   
 9          Tom Latham         tom-latham           T Latham              WK   
 10   Mitchell Santner   mitchell-santner          M Santner              AR   
 11         Matt Henry       

In [22]:
# creating match_id feature in every dataframe
for key in all_dataframe:
    all_dataframe[key]['match_id'] = match_id
    all_dataframe[key]['team'] = 'away'

In [23]:
all_dataframe['parent_df']

,supportStaff_0_name,supportStaff_0_role,supportStaff_0_id,supportStaff_0_country_alpha2,supportStaff_0_country_alpha3,supportStaff_0_country_name,supportStaff_0_country_slug,supportStaff_1_name,supportStaff_1_role,supportStaff_1_id,...,playerColor_primary,playerColor_number,playerColor_outline,playerColor_fancyNumber,goalkeeperColor_primary,goalkeeperColor_number,goalkeeperColor_outline,goalkeeperColor_fancyNumber,match_id,team
0,Mark Greatbatch,batting-coach,124,NZ,NZL,New Zealand,new-zealand,Andre Adams,bowling-coach,281,...,000000,222226,808080,222226,000000,222226,808080,222226,13569002,away


In [24]:
all_dataframe['players']

,player_name,player_slug,player_shortName,player_position,player_jerseyNumber,player_height,player_userCount,player_id,player_country_alpha2,player_country_alpha3,...,player_cricketPlayerInfo_batting,player_cricketPlayerInfo_bowling,teamId,position,substitute,player_firstName,player_lastName,captain,match_id,team
0,Daryl Mitchell,daryl-mitchell,Daryl Mitchell,AR,75,183,98,786833,NZ,NZL,...,Right,Right-arm medium,241846,AR,False,NaN,NaN,NaN,13569002,away
1,Glenn Phillips,glenn-phillips,Glenn Phillips,AR,23,NaN,106,872201,NZ,NZL,...,Right,Off break,216006,AR,False,NaN,NaN,NaN,13569002,away
2,Kane Williamson,kane-williamson,K Williamson,BM,22,174,290,786552,NZ,NZL,...,Right,Off break,187778,BM,False,NaN,NaN,NaN,13569002,away
3,Kyle Jamieson,kyle-jamieson,Kyle Jamieson,B,12,203,11,920980,NZ,NZL,...,Right,Right-arm fast medium,241846,B,False,NaN,NaN,NaN,13569002,away
4,Michael Bracewell,michael-bracewell,Michael Bracewell,AR,4,178,15,915980,NZ,NZL,...,Left,Off break,241847,AR,False,NaN,NaN,NaN,13569002,away
5,Nathan Smith,nathan-smith,N Smith,AR,NaN,NaN,10,967474,NZ,NZL,...,Right,Right-arm medium fast,241847,AR,False,NaN,NaN,NaN,13569002,away
6,Rachin Ravindra,rachin-ravindra,R Ravindra,AR,8,NaN,253,974426,NZ,NZL,...,Left,Slow left-arm orthodox,187777,AR,False,NaN,NaN,NaN,13569002,away
7,Will O'Rourke,william-orourke,Will O'Rourke,B,NaN,NaN,13,1212990,NZ,NZL,...,Right,Right-arm fast,241846,B,False,NaN,NaN,NaN,13569002,away
8,Will Young,will-young,W Young,BM,32,188,32,916142,NZ,NZL,...,Right,Off break,241848,BM,False,NaN,NaN,NaN,13569002,away
9,Tom Latham,tom-latham,T Latham,WK,48,173,31,786550,NZ,NZL,...,Left,Right-arm medium,241846,WK,False,NaN,NaN,NaN,13569002,away


# Saving

In [25]:
output_file = "away.xlsx"  # Define output Excel file

for i, (key, df) in enumerate(all_dataframe.items()):
    print(f"Saving data for {key} in {output_file}")
    
    # If it's the first DataFrame, write with mode 'w' (write mode) to create the file
    # For the rest, use mode 'a' (append) without re-writing the file
    with pd.ExcelWriter(output_file, engine="openpyxl", mode='w' if i == 0 else 'a') as writer:
        df.to_excel(writer, sheet_name=key[:31], index=False)  # Ensure sheet name is within 31 chars

print(f"All DataFrames have been saved in {output_file}")

Saving data for players in away.xlsx
Saving data for parent_df in away.xlsx
All DataFrames have been saved in away.xlsx
